In [1]:
# !wget https://huggingface.co/datasets/mesolitica/semisupervised-corpus/resolve/main/similarity/shuffled-train.json
# !wget https://huggingface.co/datasets/mesolitica/semisupervised-corpus/resolve/main/similarity/shuffled-test.json

In [2]:
from bidirectional_mistral import MistralBiModel
from transformers import MistralPreTrainedModel
import torch
import numpy as np
import json
from typing import Optional, List
from torch import nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers.modeling_outputs import SequenceClassifierOutputWithPast

In [3]:
class MistralForSequenceClassification(MistralPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.model = MistralBiModel(config)
        self.score = nn.Linear(config.hidden_size, self.num_labels, bias=False)

        # Initialize weights and apply final processing
        self.post_init()
        
    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        transformer_outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        pooled_output = transformer_outputs[0][:, 0]
        logits = self.score(pooled_output)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)
        if not return_dict:
            output = (logits,) + transformer_outputs[2:]
            return ((loss,) + transformer_outputs) if loss is not None else output
        
        return SequenceClassifierOutputWithPast(
            loss=loss,
            logits=logits,
            past_key_values=transformer_outputs.past_key_values,
            hidden_states=transformer_outputs.hidden_states,
            attentions=transformer_outputs.attentions,
        )

In [4]:
from transformers import AutoTokenizer, AutoConfig

In [5]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/malaysian-mistral-349M-MLM-512')
# tokenizer.padding_side = 'left'
config = AutoConfig.from_pretrained('mesolitica/malaysian-mistral-349M-MLM-512')
config.problem_type = "single_label_classification"
config.label2id = {'contradiction': 0, 'entailment': 1}

In [6]:
model = MistralForSequenceClassification.from_pretrained(
    'mesolitica/malaysian-mistral-349M-MLM-512', 
    config = config,
    use_flash_attention_2 = True,
)

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in MistralForSequenceClassification is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`
Flash Attention 2.0 only supports torch.float16 and torc

In [7]:
trainable_parameters = [param for param in model.parameters() if param.requires_grad]
trainer = torch.optim.AdamW(trainable_parameters, lr = 2e-5)

In [8]:
train_X, train_Y = [], []
with open('shuffled-train.json') as fopen:
    for l in fopen:
        l = json.loads(l)
        train_X.append(l['src'])
        train_Y.append(l['label'])

In [9]:
test_X, test_Y = [], []
with open('shuffled-test.json') as fopen:
    for l in fopen:
        l = json.loads(l)
        test_X.append(l['src'])
        test_Y.append(l['label'])
        
len(test_X)

16037

In [10]:
batch_size = 4
epoch = 100

In [11]:
from accelerate import Accelerator
accelerator = Accelerator(mixed_precision = 'bf16')

In [12]:
device = accelerator.device
model, trainer = accelerator.prepare(model, trainer)

In [ ]:
from tqdm import tqdm

best_dev_acc = -np.inf
patient = 1
current_patient = 0

for e in range(epoch):
    pbar = tqdm(range(0, len(train_X), batch_size))
    losses = []
    for i in pbar:
        trainer.zero_grad()
        x = train_X[i: i + batch_size]
        y = np.array(train_Y[i: i + batch_size])
        
        padded = tokenizer(x, truncation = True, padding = True, return_tensors = 'pt', max_length = 2048)
        padded['labels'] = torch.from_numpy(y)
        for k in padded.keys():
            padded[k] = padded[k].cuda()

        padded.pop('token_type_ids', None)

        o = model(**padded, use_cache = False)
        loss = o.loss
        
        accelerator.backward(loss)

        grad_norm = torch.nn.utils.clip_grad_norm_(trainable_parameters, 1.0)
        trainer.step()
            
        losses.append(float(loss))
        pbar.set_postfix(loss = float(loss))
        
        
    dev_predicted = []
    for i in range(0, len(test_X[:10000]), batch_size):
        x = test_X[i: i + batch_size]
        y = np.array(test_Y[i: i + batch_size])
        padded = tokenizer(x, truncation = True, padding = True, return_tensors = 'pt', max_length = 2048)
        padded['labels'] = torch.from_numpy(y)
        for k in padded.keys():
            padded[k] = padded[k].cuda()
            
        padded.pop('token_type_ids', None)
        
        o = model(**padded, use_cache = False)
        loss = o.loss
        pred = o.logits
        
        dev_predicted.append((pred.argmax(axis = 1).detach().cpu().numpy() == y).mean())
        
    dev_predicted = np.mean(dev_predicted)
    
    print(f'epoch: {e}, loss: {np.mean(losses)}, dev_predicted: {dev_predicted}')
    
    if dev_predicted >= best_dev_acc:
        best_dev_acc = dev_predicted
        current_patient = 0
        model.save_pretrained('64M')
    else:
        current_patient += 1
    
    if current_patient >= patient:
        break

  2%|█▎                                                           | 5665/273391 [07:47<5:38:35, 13.18it/s, loss=0.688]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  9%|█████▋                                                      | 25886/273391 [35:34<5:46:56, 11.89it/s, loss=0.467]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 17%|█████████▊                                                | 46298/273391 [1:03:37<5:18:08, 11.90it/s, loss=0.

In [15]:
!nvidia-smi

Wed Apr 24 12:08:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090 Ti     On  | 00000000:01:00.0 Off |                  Off |
| 46%   53C    P2             169W / 480W |  21145MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
real_Y = []
for i in tqdm(range(0, len(test_X), batch_size)):
    x = test_X[i: i + batch_size]
    y = np.array(test_Y[i: i + batch_size])
    padded = tokenizer(x, padding = 'longest', return_tensors = 'pt')
    padded['labels'] = torch.from_numpy(y)
    for k in padded.keys():
        padded[k] = padded[k].cuda()
    
    padded.pop('token_type_ids', None)
    o = model(**padded, use_cache = False)
    loss = o.loss
    pred = o.logits
    real_Y.extend(pred.argmax(axis = 1).detach().cpu().numpy().tolist())

100%|█████████████████████████████████████████████████████████████████████████████| 4010/4010 [01:20<00:00, 49.60it/s]


In [21]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, test_Y[:len(real_Y)],
        digits = 5
    )
)

              precision    recall  f1-score   support

           0    0.71505   0.77944   0.74586      7073
           1    0.81266   0.75491   0.78272      8964

    accuracy                        0.76573     16037
   macro avg    0.76385   0.76718   0.76429     16037
weighted avg    0.76961   0.76573   0.76646     16037



In [22]:
len(real_Y)

16037

In [23]:
len(test_Y)

16037